<a href="https://colab.research.google.com/github/aleeshajaganath/Images-and-video/blob/master/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*gdrive*

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My Drive

/content/gdrive/My Drive


In [0]:
!ls

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import matplotlib.image as mpimg
import cv2
import os
import csv
import numpy as np
from PIL import Image
for file in fileList:
    #print(file)
   #
   try:
    imgfile = Image.open(file) # 
    #print(i)
    i=i+1#    imgfile.show()
    img = load_img(file, grayscale=False, target_size=(64, 64,3))
    
                  
    img = img_to_array(img)
                  
    #
    #img = img.reshape(64, 64,3)
    #150, 150, 3
                  
    img = img.astype('float32')
                  
    #
    img = img / 255.0
                  
    value = img.flatten()
                  
    with open("validation.csv", 'a') as f:
                      
        writer = csv.writer(f)
        writer.writerow(value)
      
   except OSError as e:
        print("last file is"+file)

Using TensorFlow backend.


**trainX.csv**

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import matplotlib.image as mpimg
import cv2
import os
import csv
import numpy as np
from PIL import Image
for file in fileList:
    #print(file)
   #
   try:
    imgfile = Image.open(file) # 
    #print(i)
    i=i+1#    imgfile.show()
    img = load_img(file, grayscale=False, target_size=(64, 64,3))
    
                  
    img = img_to_array(img)
                  
    #
    #img = img.reshape(64, 64,3)
    #150, 150, 3
                  
    img = img.astype('float32')
                  
    #
    img = img / 255.0
                  
    value = img.flatten()
                  
    with open("training_check.csv", 'a') as f:
                      
        writer = csv.writer(f)
        writer.writerow(value)
      
   except OSError as e:
        print("last file is"+file)

    
    
    

checking

RESNET50

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 19 00:21:40 2019

@author: user
"""
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
#%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut]) 
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X



def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1),padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1,1), strides = (1,1),name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)conv_name_base + '1'
    X_shortcut=Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
   
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X



def ResNet50(input_shape = (64, 64, 3), classes = 224):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128,128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128,128, 512], stage=3, block='b')
    X =identity_block(X, 3, [128,128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128,128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256,256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256,256, 1024], stage=4, block='b')
    X =identity_block(X, 3, [256,256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256,256, 1024], stage=4, block='d')
    X =identity_block(X, 3, [256,256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256,256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512,512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512,512, 2048], stage=5, block='b')
    X =identity_block(X, 3, [512,512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
!ls

In [0]:
Y_train= pd.read_csv('trainY_1.csv',sep=',')
Y_train

In [0]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import math
from keras.utils import to_categorical

Y_test= pd.read_csv('ta1.csv',sep=',')
X_test= pd.read_csv('validationX1.csv',sep=',')
Y_train= pd.read_csv('ta.txt',sep=',')
X_train= pd.read_csv('trainX1.csv',sep=',')



#X_train, Y_train, X_test, Y_test_orig, classes = load_dataset()

#print(X_train_orig.shape)
# Normalize image vectors
#X_train = X_train_orig/255.
#X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
#Y_train = to_categorical(Y_train_orig).reshape((1080, 6))
#Y_test = to_categorical(Y_test_orig).reshape((120, 6))

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 14719
number of test examples = 2999
X_train shape: (14719, 12288)
Y_train shape: (14718, 224)
X_test shape: (2999, 12288)
Y_test shape: (2999, 224)


In [0]:
import pandas as pd
Y_train= pd.read_csv('ta.txt',sep=',')
Y_test= pd.read_csv('ta1.csv',sep=',')
X_train=X_train[:14718]

In [0]:
X_test= pd.read_csv('validationX1.csv',sep=',')

X_train= pd.read_csv('trainX1.csv',sep=',')

In [0]:
Y_train=Y_train.to_numpy()
Y_test=Y_test.to_numpy()

In [0]:
import numpy as np
X_train=np.array(X_train[:][:])
Y_test=np.array(Y_test[:][:])
Y_train=np.array(Y_train[:][:])

In [0]:

X_test=np.array(X_test[:][:])
Y_test=np.array(Y_test[:][:])
X_test=X_test.reshape([2999,64,64 ,3])
X_train=X_train.reshape([14718,64,64 ,3])

In [0]:
from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY= train_test_split(X_train,Y_train,test_size=0.20)


In [0]:
model = ResNet50(input_shape = (64, 64, 3), classes = 224)

In [0]:
#Ytest.shape
print(Y_test.shape,X_test.shape,Y_train.shape,X_train.shape)

(2999, 224) (2999, 64, 64, 3) (14718, 224) (14718, 64, 64, 3)


In [0]:
X_test=X_test.reshape([2999,64,64 ,3])
X_train=X_train.reshape([14718,64,64 ,3])

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(X_train , Y_train , epochs = 2, batch_size = 32)

Epoch 1/2
14718/14718 [==============================] - 157s 11ms/step - loss: 0.1290 - acc: 0.9741
Epoch 2/2
14718/14718 [==============================] - 139s 9ms/step - loss: 0.1225 - acc: 0.9742


In [0]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

2999/2999 [==============================] - 6s 2ms/step
Loss = 0.11999377537246703
Test Accuracy = 0.9743396608421666


**Using pre trained model**